In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,BatchNormalization,Dropout
from keras.optimizers import Adam,SGD

In [2]:
train_data = keras.utils.image_dataset_from_directory(
    directory='images/train',
    labels='inferred',
    label_mode='int',
    batch_size = 10,
    image_size=(256,256)
)
test_data = keras.utils.image_dataset_from_directory(
    directory='images/val',
    labels='inferred',
    label_mode='int',
    batch_size = 10,
    image_size=(256,256)
)

Found 192 files belonging to 2 classes.
Found 30 files belonging to 2 classes.


In [3]:
def normalize(image,label):
    image = tf.cast(image/255,tf.float32)
    return image,label
train_data = train_data.map(normalize)
test_data = test_data.map(normalize)

In [8]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='same',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 256, 256, 32)      896       
                                                                 
 batch_normalization_3 (Batc  (None, 256, 256, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 126, 126, 64)      18496     
                                                                 
 batch_normalization_4 (Batc  (None, 126, 126, 64)     256       
 hNormalization)                                                 
                                                      

In [10]:
opt = SGD(learning_rate=0.001,momentum=0.9)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_data,epochs=20,validation_data=test_data)

Epoch 1/20
20/20 [==============================] - 3s 114ms/step - loss: 0.1726 - accuracy: 0.9375 - val_loss: 2.3338 - val_accuracy: 0.6667
Epoch 2/20
20/20 [==============================] - 2s 113ms/step - loss: 0.2975 - accuracy: 0.8906 - val_loss: 2.9367 - val_accuracy: 0.6667
Epoch 3/20
20/20 [==============================] - 2s 111ms/step - loss: 0.2169 - accuracy: 0.9271 - val_loss: 3.0912 - val_accuracy: 0.6667
Epoch 4/20
20/20 [==============================] - 3s 159ms/step - loss: 0.1081 - accuracy: 0.9688 - val_loss: 4.0204 - val_accuracy: 0.6667
Epoch 5/20
20/20 [==============================] - 4s 159ms/step - loss: 0.1169 - accuracy: 0.9583 - val_loss: 4.6770 - val_accuracy: 0.6667
Epoch 6/20
20/20 [==============================] - 3s 143ms/step - loss: 0.1058 - accuracy: 0.9531 - val_loss: 4.8853 - val_accuracy: 0.6667
Epoch 7/20
20/20 [==============================] - 3s 130ms/step - loss: 0.0617 - accuracy: 0.9792 - val_loss: 5.3333 - val_accuracy: 0.6667
Epoch 

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
path = 'images/test/room.jpg'
img = cv2.imread(path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(256,256))
plt.imshow(img)
plt.axis('off')
img = img.reshape((1,256,256,3))
pred = "Clean" if model.predict(img)[0] >= 0.5 else "Messy"
plt.title(pred)
plt.show()

In [ ]:
fig,axes = plt.subplots(2,5,figsize=(16,8))
k=0

for i in range(2):
    for j in range(5):
        path = f'images/test/{k}.png'
        k+=1
        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(256,256))
        axes[i][j].imshow(img)
        axes[i][j].axis('off')
        img = img.reshape((1,256,256,3))
        img = tf.cast(img,tf.float32)
        pred = "Clean" if model.predict(img)[0] >= 0.5 else "Messy"
        axes[i][j].set_title(pred)

# Using Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# create data generators
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
 width_shift_range=0.2, height_shift_range=0.2,rotation_range=10,zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
train = train_datagen.flow_from_directory('images/train/',
    class_mode='binary', batch_size=10, target_size=(256, 256))
test = test_datagen.flow_from_directory('images/val/',
    class_mode='binary', batch_size=10, target_size=(256, 256))

In [ ]:
model = Sequential()
model.add(Conv2D(16,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train,validation_data=test,epochs=25)

In [ ]:
fig,axes = plt.subplots(2,5,figsize=(16,8))
k=0

for i in range(2):
    for j in range(5):
        path = f'images/test/{k}.png'
        k+=1
        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(256,256))
        axes[i][j].imshow(img)
        axes[i][j].axis('off')
        img = img.reshape((1,256,256,3))
        img = tf.cast(img,tf.float32)
        pred = "Clean" if model.predict(img)[0] >= 0.5 else "Messy"
        axes[i][j].set_title(pred)